In [1]:
import geopandas as gpd
import pandas as pd
from scipy.spatial.distance import euclidean

# Read the GeoJSON file and filter out all polygons in Shenzhen city
raw_gdf = gpd.read_file('sgh_geolist.geojson')
shenzhen_gdf = raw_gdf[raw_gdf['CSMC'] == '深圳市']

# Calculate the centroid for each polygon
shenzhen_gdf['centroid'] = shenzhen_gdf['geometry'].centroid

# Record JDBH number separately
jdbh_list = shenzhen_gdf['JDBH'].tolist()

# Read OD data from CSV file and filter out shenzhen flow according to JDBH
od_df = pd.read_csv('SGH_movment_Street_level.csv')
shenzhen_od_df = od_df[od_df['JDBH_x'].isin(jdbh_list) & od_df['JDBH_y'].isin(jdbh_list)]

# Converted to Tij matrix
Tij_matrix = shenzhen_od_df.pivot(index='JDBH_x', columns='JDBH_y', values='flow')
Tij_matrix.fillna(0, inplace=True)

# Initialize the cij matrix
cij_matrix = pd.DataFrame(index=jdbh_list, columns=jdbh_list)

# Calculate the Euclidean distance between centroids to fill the cij matrix
for i in jdbh_list:
    for j in jdbh_list:
        if i != j:
            centroid_i = shenzhen_gdf[shenzhen_gdf['JDBH'] == i]['centroid'].iloc[0]
            centroid_j = shenzhen_gdf[shenzhen_gdf['JDBH'] == j]['centroid'].iloc[0]
            
            x1, y1 = centroid_i.x, centroid_i.y
            x2, y2 = centroid_j.x, centroid_j.y
            
            cij_matrix.loc[i, j] = euclidean([x1, y1], [x2, y2])
        else:
            cij_matrix.loc[i, j] = 0

# Save the converted matrices as CSV files
Tij_matrix.to_csv('shenzhen_Tij_matrix.csv', index_label='JDBH')
cij_matrix.to_csv('shenzhen_cij_matrix.csv', index_label='JDBH')


d:\Software\anaconda\envs\geospatial\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.8.0-CAPI-1.13.1). Conversions between both will be slow.
  warnings.warn(
C:\Users\Xinyi Liu\AppData\Local\Temp\ipykernel_8488\26460367.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. Howev

In [7]:
import numpy as np
import pandas as pd

class SimpleModel:
    def __init__(self, Cij, TObs):
        self.Cij = np.array(Cij, dtype=float)  # Convert Cij to numpy array
        self.TObs = TObs.values if isinstance(TObs, pd.DataFrame) else TObs  # Extract values from DataFrame if necessary
        self.Beta = 1.0  # Initial value of Beta
    
    def run(self):
        N = self.TObs.shape[0]  # Number of rows in TObs
        B = np.ones(N)  # Array of ones with length N
        DjObs = self.TObs.sum(axis=0)  # Sum of each column in TObs
        OiObs = self.TObs.sum(axis=1)  # Sum of each row in TObs
        Tij = np.zeros_like(self.TObs)  # Initialize Tij with zeros
        converged = False  # Flag to check if the model has converged

        iteration_count = 0  # Counter for the number of iterations
        beta_min = 1e-5
        while not converged:
            print(f"Iteration {iteration_count}: Beta = {self.Beta}")
            iteration_count += 1
            
            expBetaCij = np.exp(-self.Beta * self.Cij)  # Exponential of -Beta * Cij
            for i in range(N):  # Loop over each row in TObs
                denom = np.sum(DjObs * expBetaCij[i, :])  # Denominator for the Tij formula
                Tij[i, :] = OiObs[i] * (B * DjObs * expBetaCij[i, :] / denom)  # Calculate Tij
            
            CBarPred = np.sum(Tij * self.Cij) / np.sum(Tij)  # Predicted CBar
            CBarObs = np.sum(self.TObs * self.Cij) / np.sum(self.TObs)  # Observed CBar
            print(f"CBarPred = {CBarPred}, CBarObs = {CBarObs}")
            delta = abs(CBarPred - CBarObs)  # Difference between predicted and observed CBar

            if CBarPred == 0.0:  # Prevent division by zero and Beta turning to zero
                self.Beta = self.Beta * 0.9  # Reduce Beta
                continue  # Skip to the next iteration
            
            if delta / CBarObs < 0.001:  # Convergence criterion
                converged = True
            else:
                new_beta = self.Beta * CBarPred / CBarObs  # Update Beta
                self.Beta = max(new_beta, beta_min) 
    
        self.TPred = Tij  # Set Tij as the predicted values


Cij = cij_matrix.applymap(pd.to_numeric)
assert not Cij.isnull().any().any(), "Cij contains NaN values"
assert not (Cij == np.inf).any().any(), "Cij contains infinity values"

Cij = cij_matrix
TObs = Tij_matrix

model = SimpleModel(Cij, TObs)  # Create model instance
model.run()  # Run the model
print(model.TPred)  # Print the predicted values


Iteration 0: Beta = 1.0
CBarPred = 0.0, CBarObs = 6885.2480872560445
Iteration 1: Beta = 0.9
CBarPred = 0.0, CBarObs = 6885.2480872560445
Iteration 2: Beta = 0.81
CBarPred = 0.0, CBarObs = 6885.2480872560445
Iteration 3: Beta = 0.7290000000000001
CBarPred = 0.0, CBarObs = 6885.2480872560445
Iteration 4: Beta = 0.6561000000000001
CBarPred = 0.0, CBarObs = 6885.2480872560445
Iteration 5: Beta = 0.5904900000000002
CBarPred = 9.99128656128483e-295, CBarObs = 6885.2480872560445
Iteration 6: Beta = 1e-05
CBarPred = 18686.865604612285, CBarObs = 6885.2480872560445
Iteration 7: Beta = 2.714043904852164e-05
CBarPred = 17117.554363910804, CBarObs = 6885.2480872560445
Iteration 8: Beta = 6.747439380337906e-05
CBarPred = 13825.126977218044, CBarObs = 6885.2480872560445
Iteration 9: Beta = 0.0001354841612416456
CBarPred = 9350.727709316321, CBarObs = 6885.2480872560445
Iteration 10: Beta = 0.00018399852621732034
CBarPred = 7092.601119353807, CBarObs = 6885.2480872560445
Iteration 11: Beta = 0.00018

In [11]:
formatted_array = np.vectorize('{:.2f}'.format)(model.TPred)
formatted_df = pd.DataFrame(formatted_array)
formatted_df.to_csv('shenzhen_Tij_Predmatrix.csv')